In [3]:
!pip install -e .. -U

Obtaining file:///Users/mathiashaugestad/Documents/Projects/noun-decomposition/noun-decomposition
  Preparing metadata (setup.py) ... done
  Attempting uninstall: noun-decomposition
    Found existing installation: noun-decomposition 0.0.1
    Uninstalling noun-decomposition-0.0.1:
      Successfully uninstalled noun-decomposition-0.0.1
  Running setup.py develop for noun-decomposition


In [1]:
from Secos import Decomposition
from tqdm import tqdm
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
Decomposition.download_model('de')

100%|██████████| 24.0M/24.0M [00:06<00:00, 3.96MB/s]

Download completed!


In [2]:
model = Decomposition.load_model('de')

In [3]:
model

DecompoundingModel(German)

In [4]:
secos = Decomposition(model)

In [6]:
secos.decompose("Bundesfinanzministerium")

['bundes', 'finanz', 'ministerium']

In [9]:
with open('./german_wictionary.txt','r') as f:
    evaluation_data = {}
    for i, line in enumerate(f.read().splitlines()):
        word, splits = line.split('\t')[0:2]
        evaluation_data[word] = set(splits.split('-'))

In [11]:
"""\
Precision and Recall are defined as usual: Precision is obtained by dividing
the number of correct splits by the number of splits taken by the method, recall is
the number of correct splits divided by the total number of splits in the evaluation
set. We further report results in the F1 measure, which is the harmonic mean of
Precision and Recall.\
"""

'Precision and Recall are defined as usual: Precision is obtained by dividing\nthe number of correct splits by the number of splits taken by the method, recall is\nthe number of correct splits divided by the total number of splits in the evaluation\nset. We further report results in the F1 measure, which is the harmonic mean of\nPrecision and Recall.'

In [25]:
results = []
for key, item in tqdm(evaluation_data.items()):
    preds = set(secos.decompose(key))
    results.append(
        {'word': key,
         'predicted': preds,
         'correct_splits': len(preds.intersection(item)),
         'compound_wrong_split': len(item.difference(preds)),
         'compound_not_split': len(preds.difference(item))
        }
    )

  0%|          | 0/5688 [00:00<?, ?it/s]/Users/mathiashaugestad/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:411: RuntimeWarning: Mean of empty slice.
  return np.exp(log_a.mean(axis=axis))
/Users/mathiashaugestad/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5688/5688 [00:00<00:00, 40441.69it/s]


In [26]:
df = pd.DataFrame(results)
df

,word,predicted,correct_splits,compound_wrong_split,compound_not_split
0,Aalfang,{Aalfang},0,2,1
1,Aalfell,{Aalfell},0,2,1
2,Aalfett,{Aalfett},0,2,1
3,Aalhaut,{Aalhaut},0,2,1
4,Aalmolch,{Aalmolch},0,2,1
...,...,...,...,...,...
5683,Zwetschgendatschi,{Zwetschgendatschi},0,2,1
5684,Zwiebelfisch,"{fisch, Zwiebel}",2,0,0
5685,Zwischenprodukt,"{produkt, Zwischen}",2,0,0
5686,Zwischenschicht,"{schicht, Zwischen}",2,0,0


In [31]:
df.correct_splits.sum() / (df.correct_splits.sum() + df.query('compound_not_split == 0').compound_wrong_split.sum())

1.0

In [8]:
p = 1.0*k[0]/(1.0*k[0]+k[1])
r = 1.0*k[0]/(1.0*k[0]+k[1]+k[2])
f = 2*p*r/(p+r)

,word,correct_split,predicted,correct
2213,Jammertal,"[Jammer, tal]",[Jammertal],0
515,Betriebssystemkern,"[Betriebssystem, kern]",[Betriebssystemkern],0
1,Aalfell,"[Aal, fell]",[Aalfell],0
1564,Gammastrahl,"[Gamma, strahl]",[Gammastrahl],0
2531,Kopfgeld,"[Kopf, geld]",[Kopfgeld],0
4775,Strahlenschutzkleidung,"[Strahlenschutz, kleidung]","[Strahlen, schutz, kleidung]",0
3992,Roheisen,"[Roh, eisen]",[Roheisen],0
5253,Waldeinsamkeit,"[Wald, einsamkeit]",[Waldeinsamkeit],0
301,Atomenergie,"[Atom, energie]",[Atomenergie],0
5613,Zinkselenid,"[Zink, selenid]",[Zinkselenid],0


In [9]:
# How many words does it try to split?
recall = df.predicted.apply(lambda x: len(x) > 1).sum() / len(df)
recall

0.6216596343178622

In [10]:
# When it splits a word, how often does it get it right?
precision = df[df.predicted.apply(lambda x: len(x) > 1)].correct.sum() / len(df[df.predicted.apply(lambda x: len(x) > 1)])
precision

0.7890271493212669

In [11]:
f1 = (2 * precision * recall) / (precision + recall)
f1

0.6954149351971267

In [12]:
df['gold'] = df.correct_split.apply(lambda x: "-".join(x))
df['preds'] = df.predicted.apply(lambda x: "-".join(x))

df.head()

,word,correct_split,predicted,correct,gold,preds
0,Aalfang,"[Aal, fang]",[Aalfang],0,Aal-fang,Aalfang
1,Aalfell,"[Aal, fell]",[Aalfell],0,Aal-fell,Aalfell
2,Aalfett,"[Aal, fett]",[Aalfett],0,Aal-fett,Aalfett
3,Aalhaut,"[Aal, haut]",[Aalhaut],0,Aal-haut,Aalhaut
4,Aalmolch,"[Aal, molch]",[Aalmolch],0,Aal-molch,Aalmolch


In [12]:
df[['preds', 'gold']].to_csv('eval_file_generated_70M', sep='\t', header=False, index=False)

In [17]:
with open('compounds_wictionary', 'w') as f:
    for line in df.word.tolist():
        f.write(line + '\n')